In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from fmeval.data_loaders.data_config import DataConfig
from fmeval.model_runners.bedrock_model_runner import BedrockModelRunner
from fmeval.constants import MIME_TYPE_JSONLINES
from fmeval.eval_algorithms.faithfulness import Faithfulness

import json

/opt/conda/envs/rageval_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-16 20:18:56,092	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ubuntu/.config/sagemaker/config.yaml


2024-07-16 20:18:57,071	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [3]:
config = DataConfig(
    dataset_name="rag-dataset-refs-test-100",
    dataset_uri="rag-dataset-refs-test-100.jsonl",
    dataset_mime_type=MIME_TYPE_JSONLINES,
    model_input_location="question_with_prompt",
    model_output_location="answer",
    target_context_location="target_context",
)

In [4]:
import os
os.environ['AWS_DEFAULT_REGION'] = 'us-west-2'

In [5]:
import os
eval_dir = "results-eval-faithfulness"
curr_dir = os.getcwd()
eval_results_path = os.path.join(curr_dir, eval_dir) + "/"
os.environ["EVAL_RESULTS_PATH"] = eval_results_path
if os.path.exists(eval_results_path):
    print(f"Directory '{eval_results_path}' exists.")
else:
    os.mkdir(eval_results_path)

Directory '/home/ubuntu/efs/clarify_rag/rag_eval/results-eval-faithfulness/' exists.


In [6]:
claude3_sonnet_model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
claude3_haiku_model_id = "anthropic.claude-3-haiku-20240307-v1:0"

In [56]:
# Taken from https://aws.amazon.com/bedrock/pricing/
claude3_cost = {
    claude3_haiku_model_id: {
        'pp1k_input_tokens': 0.00025,
        'pp1k_output_tokens': 0.00125,
    },
    claude3_sonnet_model_id: {
        'pp1k_input_tokens': 0.003,
        'pp1k_output_tokens': 0.015,
    },
}

In [8]:
content_template = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 10000,
        "temperature": 0.0,
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": -1},
                ],
            }
        ],
    }
content_template = json.dumps(content_template)
content_template = content_template.replace('-1', '$prompt')
content_template

'{"anthropic_version": "bedrock-2023-05-31", "max_tokens": 10000, "temperature": 0.0, "messages": [{"role": "user", "content": [{"type": "text", "text": $prompt}]}]}'

In [9]:
def get_bedrock_model(model_id, output_jmespath) -> BedrockModelRunner:
    """Get default judge model

    :return: A Bedrock Model Runner with default model id.
    """
    return BedrockModelRunner(
        model_id=model_id,
        output=output_jmespath,
        content_template=content_template,
    )

In [38]:
model_id = claude3_haiku_model_id

In [25]:
judge_model = get_bedrock_model(model_id, "content[0].text")

# Crafting a prompt using Anthropic cookbook [guidelines](https://docs.anthropic.com/en/docs/build-with-claude/prompt-engineering/use-xml-tags)

In [26]:
long_form_example = """
<example>
<text>
Question: Who was Albert Einstein and what is he best known for?
Answer: Albert Einstein was a German-born theoretical physicist, widely acknowledged to be one of the greatest and most influential physicists of all time. He was best known for developing the theory of relativity, he also made important contributions to the development of the theory of quantum mechanics.
</text>
<statements>
Statement: Albert Einstein was a German-born theoretical physicist.
Statement: Albert Einstein is recognized as one of the greatest and most influential physicists of all time.
Statement: Albert Einstein was best known for developing the theory of relativity.
Statement: Albert Einstein also made important contributions to the development of the theory of quantum mechanics.
</statements>
</example>
"""

In [27]:
nli_statements_example = """
<example>
<context>
John is a student at XYZ University. He is pursuing a degree in Computer Science. He is enrolled in several courses this semester, including Data Structures, Algorithms, and Database Management. John is a diligent student and spends a significant amount of time studying and completing assignments. He often stays late in the library to work on his projects.
</context>
<statements>
Statement: John is majoring in Biology.
Statement: John is taking a course on Artificial Intelligence.
Statement: John is a dedicated student.
Statement: John has a part-time job.
Statement: John is interested in computer programming.
</statements>
<judgements>
<judgement>
Statement: John is majoring in Biology.
Verdict: No
Explanation: John's major is explicitly mentioned as Computer Science. There is no information suggesting he is majoring in Biology.  Verdict: No.
</judgement>
<judgement>
Statement: John is taking a course on Artificial Intelligence.
Verdict: No
Explanation: The context mentions the courses John is currently enrolled in, and Artificial Intelligence is not mentioned. Therefore, it cannot be deduced that John is taking a course on AI. Verdict: No.
</judgement>
<judgement>
Statement: John is a dedicated student.
Verdict: Yes
Explanation: The prompt states that he spends a significant amount of time studying and completing assignments. Additionally, it mentions that he often stays late in the library to work on his projects, which implies dedication. Verdict: Yes.
</judgement>
<judgement>
Statement: John has a part-time job.
Verdict: No
Explanation: There is no information given in the context about John having a part-time job. Therefore, it cannot be deduced that John has a part-time job.  Verdict: No.
</judgement>
<judgement>
Statement: John is interested in computer programming.
Verdict: Yes
Explanation: The context states that John is pursuing a degree in Computer Science, which implies an interest in computer programming. Verdict: Yes.
</judgement>
</judgements>
</example>
"""

In [28]:
long_form_prompt_template = f"""Human:
<text>
Question: $question
Answer: $answer
</text>

You are given a question and answer within the <text> tags above.
Your task is to break down the answer into one or more simple, meaningful and unique statements.
Your statements should contain all the details and facts present in the answer.
Please do not generate redundant statements.
You should start each statement with the tag "Statement: ". Place your statements within the <statements> tags.
To help you understand the task and the output format, you are given an example below within the <example> tags.

{long_form_example}

Assistant:
"""

In [29]:
nli_statements_prompt_template = f"""Human:
<text>
<context>
$context
</context>
<statements>
$statements
</statements>
</text>

You are given a context within <context> tags and some statements within <statements> tags above.
For each given statement, your task is to judge if the statement can be inferred from the given context.
If a statement can be inferred from the given context, output your judgement as "Verdict: yes". Else, output "Verdict: no".
For each judgement also provide a simple and short explanation. Place your judgements within <judgements> tags.
To help you understand the task and the output format, you are given an example below within the <example> tags.

{nli_statements_example}

Assistant:
"""

In [39]:
eval_algo = Faithfulness()
eval_output = eval_algo.evaluate(judge_model=judge_model,
                                 dataset_config=config,
                                 long_form_prompt_template=long_form_prompt_template,
                                 nli_statements_prompt_template=nli_statements_prompt_template,
                                 num_records=100,
                                 save=True)

2024-07-17 03:01:25,181	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2024-07-16_20-19-30_230780_1258974/logs/ray-data
2024-07-17 03:01:25,182	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> AllToAllOperator[Repartition]

                                                  
2024-07-17 03:01:25,874	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2024-07-16_20-19-30_230780_1258974/logs/ray-data
2024-07-17 03:01:25,875	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> ActorPoolMapOperator[Map(GeneratePrompt)]
                                                                                                                   2024-07-17 03:01:39,380	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2024-07-16_20-19-30_230780_1258974/logs/ray-data
2024-07-17 03:01:39,380

In [50]:
eval_output

[EvalOutput(eval_name='faithfulness', dataset_name='rag-dataset-refs-test-100', dataset_scores=[EvalScore(name='faithfulness', value=0.9849047619047618, error=None)], prompt_template=None, category_scores=None, output_path='/home/ubuntu/efs/clarify_rag/rag_eval/results-eval-faithfulness/faithfulness_rag-dataset-refs-test-100.jsonl', error=None)]

In [51]:
# Create a Pandas DataFrame to visualize the results
import pandas as pd
import json

data = []
with open(os.path.join(eval_results_path, "faithfulness_rag-dataset-refs-test-100.jsonl"), "r") as file:
    for line in file:
        data.append(json.loads(line))
df = pd.DataFrame(data)
df['eval_algo'] = df['scores'].apply(lambda x: x[0]['name'])
df['eval_score'] = df['scores'].apply(lambda x: x[0]['value'])
df

,model_input,model_output,target_context,raw_verdicts,raw_statements,statements_input_tokens,statements_output_tokens,input_tokens,output_tokens,scores,eval_algo,eval_score
0,Who is the companion of the Tenth Doctor in th...,The companion of the Tenth Doctor in the episo...,"We'd Like To Thank You, Herbert Hoover, For Re...",<judgements>\n<judgement>\nStatement: The comp...,<statements>\nStatement: The companion of the ...,341,34,1556,78,"[{'name': 'faithfulness', 'value': 1.0}]",faithfulness,1.0
1,What event did the Salem Regional Medical Cent...,The Salem Regional Medical Center Foundation p...,"February 19, 2015 DETROIT (AP) — An appeals co...",<judgements>\n<judgement>\nStatement: The Sale...,<statements>\nStatement: The Salem Regional Me...,336,44,1793,139,"[{'name': 'faithfulness', 'value': 1.0}]",faithfulness,1.0
2,When did the author realize their blogiversary...,The author realized their blogiversary was on ...,"HAPPY belated BLOGIVERSARY!\nFebruary 7, 2006\...",<judgements>\n<judgement>\nStatement: The auth...,<statements>\nStatement: The author realized t...,325,27,1772,87,"[{'name': 'faithfulness', 'value': 1.0}]",faithfulness,1.0
3,What is the main issue with using two Smart Ob...,The main issue with using two Smart Objects in...,In the previous post I cited a tool for Photos...,<judgements>\n<judgement>\nStatement: The main...,<statements>\nStatement: The main issue with u...,429,113,2081,281,"[{'name': 'faithfulness', 'value': 1.0}]",faithfulness,1.0
4,What are the services offered by La Petrolifer...,La Petrolifera Italo Rumena (PIR) offers stora...,"Flexibility, advanced logistic solutions and h...",<judgements>\n<judgement>\nStatement: La Petro...,<statements>\nStatement: La Petrolifera Italo ...,413,158,1614,374,"[{'name': 'faithfulness', 'value': 1.0}]",faithfulness,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,What is the new legislation about that could g...,The new legislation is about matching dollar-f...,Newly passed legislation could give Holbrook s...,<judgements>\n<judgement>\nStatement: The new ...,<statements>\nStatement: The new legislation i...,385,88,1712,242,"[{'name': 'faithfulness', 'value': 1.0}]",faithfulness,1.0
96,How do politicians use public demographics and...,Politicians use public demographics and statis...,"Yesterday, a few co-workers and myself, were s...",<judgements>\n<judgement>\nStatement: Politici...,<statements>\nStatement: Politicians use publi...,332,47,1025,139,"[{'name': 'faithfulness', 'value': 1.0}]",faithfulness,1.0
97,What are the two primary personas that need to...,The two primary personas that need to be ackno...,If you’re struggling to sell more managed serv...,<judgements>\n<judgement>\nStatement: The two ...,<statements>\nStatement: The two primary perso...,348,42,1955,100,"[{'name': 'faithfulness', 'value': 1.0}]",faithfulness,1.0
98,What does KatyKatCafe look forward to during #...,KatyKatCafe looks forward to Boogie with the B...,.\nKatyKatCafe @BunnyJeanCook Just read the sa...,<judgements>\n<judgement>\nStatement: KatyKatC...,<statements>\nStatement: KatyKatCafe looks for...,348,40,1320,154,"[{'name': 'faithfulness', 'value': 1.0}]",faithfulness,1.0


In [52]:
errors_df = df[df['eval_score'] < 1]
if len(errors_df):
    errors_df.to_csv(f'faithfulness_{model_id}_errors.csv', index=False)

In [57]:
# Compute total cost of running judge model on the dataset
token_cost = claude3_cost[model_id]
total_input_tokens = df['statements_input_tokens'].sum() + df['input_tokens'].sum()
total_output_tokens = df['statements_output_tokens'].sum() + df['output_tokens'].sum()
input_token_cost = (token_cost['pp1k_input_tokens'] * total_input_tokens) / 1000
output_token_cost = (token_cost['pp1k_output_tokens'] * total_output_tokens) / 1000
total_cost = input_token_cost + output_token_cost

In [55]:
print(f"Mean faithfulness score {df['eval_score'].mean():.4f}")
print(f"Std faithfulness score {df['eval_score'].std():.4f}")
print(f"Total cost ${total_cost:.4f}")

Mean faithfulness score 0.9849
Std faithfulness score 0.1037
Total cost $0.0917


In [37]:
df.to_csv(f'faithfulness_{model_id}_results.csv', index=False)